<img src="../Images/DSC_Logo.png" style="width: 400px;">

# Data Preprocessing of PANGAEA Datasets

This notebook prepares previously downloaded PANGAEA datasets for visualization and analysis of orca and other cetacean sightings along Polarstern cruises.  

> Note: Only the data tables are processed here. Metadata files that contain information such as copyright and citation are not handled in this workflow, but they must always be consulted for interpretation and referenced in any use of the datasets.

# 1 Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import os

# 2 Orca Datasets
## 2.1 Load Multiple Files in Dataframe

Set path to orca data folder:

In [ ]:
dataset_directory = "../Data/PANGAEA_orca_data/Datasets"

Define columns to keep:

In [ ]:
columns = ["DATE/TIME", "LATITUDE", "LONGITUDE", "Whale species", "Individuals [#]", "Event"]

We want to load and combine all PANGAEA orca datasets into a single dataframe. We have an easy life with our choosen data example, because the retrieved PANGAEA orca datasets are already standardized. That means we can combine them directly without extra preprocessing steps such as renaming columns.

In [ ]:
frames = []

for filename in os.listdir(dataset_directory):
    file_path = os.path.join(dataset_directory, filename)
    df = pd.read_csv(file_path, sep="\t", usecols=columns)
    frames.append(df)

df_orca = pd.concat(frames, ignore_index=True)

Preview df:

In [ ]:
df_orca.head()

Length of df:

In [ ]:
len(df_orca)

## 2.2 Select Orca Data

The column "Whale species" holds not just sights of Orcas but also of other whale species: 

In [ ]:
df_orca['Whale species'].unique()

However, let's assume we are only interested in 'Orcinus orca' here so we filter the dataframe:

In [ ]:
# Filter
df_orca = df_orca[df_orca['Whale species'] == 'Orcinus orca']

# Reset index
df_orca.reset_index(drop=True, inplace=True)

Remove rows where "Individuals [#]" is NaN:

In [ ]:
df_orca = df_orca.dropna(subset=['Individuals [#]']).reset_index(drop=True)

Preview df:

In [ ]:
df_orca.head()

Length of df:

In [ ]:
len(df_orca)

Our combined dataset contains 27 sightings in total.

## 2.3 Datetime Conversion

The column "DATE/TIME" contains full timestamps for each sighting in the format YYYY-MM-DD HH:MM:SS. Let's assume for our analysis, we are interested only in the year of each record. 

We first convert the column into a proper datetime format in pandas:

In [ ]:
df_orca['DATE/TIME'] = pd.to_datetime(df_orca['DATE/TIME'])

And then extract the year as a separate column:

In [ ]:
df_orca['Year'] = df_orca['DATE/TIME'].dt.year

In [ ]:
df_orca.head()

## 2.4 Save

In [ ]:
dataset_directory = "../Data/PANGAEA_orca_data/Orca_preprocessed.txt"
df_orca.to_csv(dataset_directory, sep="\t", index=False)

# 3 Master Track Datasets
## 3.1 Load Multiple Files in Dataframe 

We load multiple files in a single dataframe as in Sect. 2.1, however, now we filter out empty dataframes (caused by zipped duplicates) and manually exclude the remaining duplicate dataset for cruise PS92.

In [ ]:
dataset_directory = "../Data/PANGAEA_mastertrack_data/Datasets"
columns = ["Date/Time", "Latitude", "Longitude", "Event"]

frames = []
for filename in os.listdir(dataset_directory):
    file_path = os.path.join(dataset_directory, filename)

    try:
        df = pd.read_csv(file_path, sep="\t", usecols=columns)
    except pd.errors.EmptyDataError:
        print(f"Skipping file: {filename}")
        continue

    if filename == "PANGAEA_master_dataset_905170.txt":
        print(f"Skipping file: {filename}")
        continue

    frames.append(df)

df_mastertrack = pd.concat(frames, ignore_index=True)

print(df_mastertrack.head())

## 3.2 Select Master Tracks With Orca Data

Get the unique events from the orca dataset:

In [ ]:
orca_events = df_orca["Event"].unique()

Filter mastertrack dataset by those events:

In [ ]:
df_mastertrack = df_mastertrack[df_mastertrack["Event"].isin(orca_events)]

## 3.3 Datetime Conversion

For plotting the master tracks on a large-scale map a different datetime conversion makes sense to us: We resample the data from 10 minutes to daily means.

Convert 'Date/Time' to datetime type:

In [ ]:
df_mastertrack['Date/Time'] = pd.to_datetime(df_mastertrack['Date/Time'])

Resample daily within each Event and take the mean of numeric cols:

In [ ]:
df_mastertrack = (
    df_mastertrack
        .groupby('Event')
        .resample('D', on='Date/Time')
        .mean(numeric_only=True)
        .reset_index()   # brings Event and Date/Time back as columns
)

In [ ]:
df_mastertrack.head()

## 3.4 Save

In [ ]:
dataset_directory = "../Data/PANGAEA_mastertrack_data/Mastertracks_preprocessed.txt"
df_mastertrack.to_csv(dataset_directory, sep="\t", index=False)

# 4 Select all Sightings of Cetacean in Individual Dataset

In the preprocessing of the (other) single dataset (Sect. 7 in notebook 1 - download pangaea data), we extract all cetacean sightings from the raw datasets. Following Jungblut et al., individual species are too scarce to analyze separately, so all records of whales and dolphins are pooled into a single group. This filtered dataset provides the basis for examining the distribution of cetaceans.

Load the dataset:

In [ ]:
dataset_directory = "../Data/868991_dataset.txt"
df = pd.read_csv(dataset_directory, sep="\t", encoding="utf-8")

Inspect columns:

In [ ]:
list(df.columns)

Define cetacean (whale + dolphin + porpoise) columns:

In [ ]:
whale_columns = [
    "P. catodon",
    "M. novaeangliae",
    "B. physalus",
    "B. borealis",
    "B. acutorostrata",
    "B. brydei",
    "G. melas",
    "G. macrorhynchus",
    "Globicephala sp.",
    "Z. cavirostris",
    "Beaked whale",
    "Whale unident",
    "O. orca", # <-
    # dolphins & porpoises (also cetaceans, included by Jungblut et al.):
    "L. australis","L. albirostris","Lagenorhynchus sp.",
    "D. delphis","S. frontalis","S. coeruleoalba","S. clymene",
    "T. truncatus","G. griseus","S. bredanensis","S. longirostris",
    "Dolphins unident","P. phocoena"
]

Add pooled Whales column:

In [ ]:
df["Whales"] = df[whale_columns].sum(axis=1)

Filter dataset - Keep only rows where Whales is not 0:

In [ ]:
df = df[df["Whales"] > 0]

In [ ]:
print(df[["Date/Time", "Latitude", "Longitude", "Whales"]].head())

Save selected columns:

In [ ]:
dataset_directory = "../Data/868991_dataset_preprocessed.txt"
df[["Date/Time","Latitude","Longitude","Whales"]].to_csv(dataset_directory, sep="\t", encoding="utf-8", index=False)